# ***AMATRICE-VISSO-NORCIA 2016-2017*** seismic sequence (Central Appennines)

#### The aim is to ***create a seismic catalog using new artificial intelligence methods***, particularly ***PhaseNet*** (Zhu et al., 2019) and the ***Seisbench*** platform, which contains training datasets and other deep learning models for picking.

### Seismic Catalog Creation: The 2016 Central Italy Seismic Sequence
## 1. Introduction
The 2016 Central Italy seismic sequence is one of the most significant recent seismic crises in Europe. It began on **August 24, 2016**, with a magnitude Mw 6.0 earthquake near Amatrice, followed by several major events in the following months (e.g., Mw 5.9 on Oct 26, Mw 6.5 on Oct 30).

![Seismicity Map - Central Italy 2016](https://upload.wikimedia.org/wikipedia/commons/a/a4/30-10-2016_central_italy_ShakeMap.jpg)

*Figure: Shake map of the 2016/10/30 Mw 6.5 Central Italy sequence (source: INGV via Wikipedia*)


List of the major earthquakes of the 2016–2017 Central Italy Sequence

| Date             | Local Time (CET/CEST) | Magnitude (Mw) | Depth (km) | Epicenter Municipality          | Latitude | Longitude |
|------------------|------------------------|----------------|------------|--------------------------------|----------|-----------|
| 24 Aug 2016      | 03:36:32               | 6.0            | 8          | Accumoli                       | 42.70 N  | 13.23 E   |
| 24 Aug 2016      | 04:33:28               | 5.3            | 8          | Norcia                         | 42.79 N  | 13.15 E   |
| 26 Oct 2016      | 19:10:36               | 5.4            | 9          | Castelsantangelo sul Nera      | 42.88 N  | 13.13 E   |
| 26 Oct 2016      | 21:18:05               | 5.9            | 8          | Ussita                         | 42.91 N  | 13.13 E   |
| 30 Oct 2016      | 07:40:17               | 6.5            | 9          | Norcia                         | 42.83 N  | 13.11 E   |
| 18 Jan 2017      | 10:25:40               | 5.1            | 10         | Montereale                    | 42.55 N  | 13.28 E   |
| 18 Jan 2017      | 11:14:09               | 5.5            | 10         | Capitignano                   | 42.53 N  | 13.28 E   |
| 18 Jan 2017      | 11:25:23               | 5.4            | 9          | Pizzoli                       | 42.50 N  | 13.28 E   |
| 18 Jan 2017      | 14:33:36               | 5.0            | 10         | Cagnano Amiterno              | 42.47 N  | 13.28 E   |

These events significantly affected Lazio, Marche, Umbria, and Abruzzo, causing extensive damage and casualties. The Mw 6.5 earthquake on October 30, 2016, was the strongest in Italy since 1980 (and it will be the case study of this tutorial).


This notebook explains **how to create a seismic catalog**, outlining each step from waveform acquisition to event publication.

## 2. Data Collection
To begin, you need:
- **Waveform data** (e.g., in MiniSEED format)
- **Station metadata** (StationXML or RESP files)
- Optional: **preliminary catalog data** (for reference)

In Italy, waveform data can be accessed via the [INGV Opendata Portal](http://terremoti.ingv.it/) or the [European Integrated Data Archive (EIDA)](https://www.orfeus-eu.org/data/eida/).


## 3. Waveform Preprocessing
Before phase picking:
- Detrend and demean the signal
- Apply band-pass filters (e.g., 1–10 Hz)
- Remove the instrument response

```python
# Example using ObsPy
from obspy import read
st = read("data/IV.SORR..HHZ.D.2016.237")
st.detrend("demean")
st.filter("bandpass", freqmin=1.0, freqmax=10.0)
st.remove_response(output="VEL")
st.plot()
```

## 4. Phase Picking
This step involves identifying P and S wave arrival times.

- **Manual picking**: Using tools like SeisGram2K or ObsPy GUI
- **Automatic picking**: Using ML models (e.g., **PhaseNet**, **EQTransformer**)

Output: Picks in a format like `.csv`, `.phs`, or QuakeML.

In this tutorial, we'll explore the **"automatic picking"** using the **new machine-learning algorithms** available such as:

### PhaseNet
- A deep learning model that predicts P and S arrival times on 3-component waveforms.
- Input: preprocessed waveforms
- Output: pick times with probabilities
- Reference: https://github.com/wayneweiqiang/PhaseNet

### SeisBench
- A unified benchmark framework for seismic deep learning models.
- Provides pre-trained models like PhaseNet, EQTransformer, etc.
- Facilitates reproducibility and comparison.
- Reference: https://github.com/seisbench/seisbench

### GaMMA
- A fast, grid-free association and relocation tool.
- Associates picks into events using a probabilistic framework.
- Efficient for real-time and offline catalog generation.
- Reference: https://github.com/AI4EPS/GaMMA


## 5. Hypocenter Location
With picks, a velocity model, and station locations, estimate the origin time and hypocenter.

Tools:
- **NonLinLoc** (probabilistic)
- **Hypo71** (linear, 1D)
- **HypoDD** (Double-Difference locations)
- **VELEST** (joint inversion of events and 1D model)

The Hypocenter location is fundamental for different analysis such as: seismic hazard map, seismic tomography, attenuation studies, seismic sources modellation, fault structures analysis etc...

## 6. Magnitude Computation
Estimate the event magnitude:
- **ML (local magnitude)**: Based on peak amplitude
- **Mw (moment magnitude)**: Based on spectral analysis or full waveform inversion

```python
# Simplified ML estimation (peak-to-peak amplitude)
amplitude = st[0].max() - st[0].min()
ml = log10(amplitude) + correction_factor
```

## 7. Catalog Compilation
Assemble a table with the following fields:
- Origin time (UTC)
- Latitude / Longitude / Depth
- Magnitude (ML, Mw, etc.)
- Number of phases used
- RMS error
- Event ID

Output formats:
- CSV / Excel
- QuakeML / XML
- Database (PostgreSQL, SQLite)

## 8. Quality Control
- Check for duplicates
- Remove false triggers
- Validate against official catalogs (e.g., INGV, USGS)

Visual checks:
- Epicenter distribution map
- Magnitude vs time (timeline)

```python
# Example visualization
import matplotlib.pyplot as plt
plt.scatter(event_times, magnitudes)
plt.xlabel("Date")
plt.ylabel("Magnitude")
plt.title("Magnitude vs Time - 2016 Central Italy")
plt.show()
```

## 9. Summary
The construction of a seismic catalog involves:
1. Data acquisition
2. Waveform preprocessing
3. Phase picking
4. Event location
5. Magnitude estimation
6. Catalog creation
7. Quality assurance

Applied to the 2016 Central Italy sequence, this workflow supports detailed seismicity analysis, hazard studies, and tectonic interpretation.



## 10. References
- INGV: http://terremoti.ingv.it/
- EIDA: https://www.orfeus-eu.org/data/eida/
- ObsPy: https://docs.obspy.org/
- Seisbench: https://github.com/seisbench/seisbench
- PhaseNet: https://github.com/wayneweiqiang/PhaseNet
- GaMMA: https://github.com/AI4EPS/GaMMA
- NonLinLoc: https://alomax.free.fr/nlloc/
- HypoDD: https://www.ldeo.columbia.edu/~felixw/hypoDD.html
- Hypoellipse: https://pubs.usgs.gov/publication/ofr79431

## 11. Install the useful packages to run the tutorial

In order to run the tutorial, please install the following packages:
```python
# install seisbench on your laptop
pip install seisbench  pyproj seaborn
#install GaMMA to provide P- and S- waves association and inititial locations
pip install git+https://github.com/AI4EPS/GaMMA.git
```